# Gun Detection with VGG16

This notebook demonstrates how to detecting guns appearing in images or video, using the pre-trained VGG16 deep learning model.

To get started, you must have the following requirements installed:

1. tensorflow - to preprocess data and work with the VGG16 model
2. numpy - to prepare data during testing
3. matplotlib - to display images in the Jupyter notebook
4. notebook - to run this notebook in a Jupyter server

It is recommended to have a virtual environment to isolate these requirements from the rest of your system. This can be done using Python's virtualenv package.

First, open a Terminal (Command Prompt on Windows) in the same folder as this notebook and create a virtual environment:

```shell
python3 -m venv env
```

Next, activate the virtual environment. For Windows users:

```shell
.\env\Scripts\activate
```

For Linux and MacOS users:

```shell
source env/bin/activate
```

Now you can safely install the above requirements in your virtual environment:

```shell
pip install tensorflow numpy matplotlib notebook
```

After installation is complete, launch the Jupyter server to edit this notebook:

```shell
jupyter notebook
```